Ершов Артем

In [1]:
import subprocess
import os
import re

from IPython.display import HTML

coverage_pattern = re.compile('[0-9]{2}x')

data_folder = 'HW6_data/'

r1 = 'illumina.100x.1.fq.gz'
r2 = 'illumina.100x.2.fq.gz'

###  Canu - риды Pacbio
Canu требует указать примерный размер генома - положим его равным 4.6Мб.

In [ ]:
for filename in os.listdir(data_folder):
    if os.path.isfile(data_folder + filename) and filename.startswith('pacbio'):
        output_folder =  filename.split('.')[0]
        coverage = re.search(coverage_pattern, filename).group(0)
        assembly_prefix = 'pb' + re.search(coverage_pattern, filename).group(0)
        cmd = [
            'canu',
            '-p', assembly_prefix,
            '-d', output_folder,
            'genomeSize=4.6m',
            'stopOnLowCoverage=1',  # иначе не обрабатывает 10х
            '-raw', '-pacbio',
            filename
        ]
        proc = subprocess.run(cmd, capture_output=True, cwd=data_folder)
        try:
            proc.check_returncode()                      
        except subprocess.CalledProcessError as e:                                                                                                   
            print(e.returncode)
            print(e.stdout.decode('utf-8'))
            print(e.stderr.decode('utf-8'))

### SPAdes - риды Illumina

In [13]:
r1 = 'illumina.100x.1.fq.gz'
r2 = 'illumina.100x.2.fq.gz'

cmd = [
    'spades.py',
    '-1', r1,
    '-2', r2,
    '-o', 'illumina100x'
]
proc = subprocess.run(cmd, capture_output=True, cwd=data_folder)
try:
    proc.check_returncode()                      
except subprocess.CalledProcessError as e:                                                                                                   
    print(e.returncode)
    print(e.stdout.decode('utf-8'))
    print(e.stderr.decode('utf-8'))

### SPAdes - гибридная сборка

In [7]:
for filename in os.listdir(data_folder):
    if os.path.isfile(data_folder + filename) and filename.startswith('pacbio'):
        coverage = re.search(coverage_pattern, filename).group(0)
        outdir = 'pb' + coverage + '+illumina100x'
        
        cmd = [
            'spades.py',
            '-1', r1,
            '-2', r2,
            '--pacbio', filename,
            '-o', outdir
        ]
        if coverage != '80x':
            continue
        try:
            proc = subprocess.run(cmd, capture_output=True, cwd=data_folder)
            proc.check_returncode()                      
        except subprocess.CalledProcessError as e:                                                                                                   
            print(e.returncode)
            print(e.stdout.decode('utf-8'))
            print(e.stderr.decode('utf-8'))

### Quast - статистики сборок

Воспользуемся веб-версией Quast.

#### Сборки из отдельных данных:

In [11]:
display(HTML('hw6_pb_illumina_reports.html'))

#### Гибридные сборки

In [13]:
display(HTML('hybrid_report.html'))

Из ридов Pacbio с покрытием 10х получилась, ожидаемо, самая некачественная сборка - геном покрыт на 80%, низкие значения метрик, много ошибок.

Между покрытием в 40х и 80х качественной разницы не наблюдается.

У Иллюмины из-за длины ридов много "коротких" контигов (хотя всё ещё меньше, чем у Pacbio 10x), и, соответственно ниже значения метрик и процет покрытого генома (один короткий контиг вообще не выравнялся), но отсутствуют мисассемблы и гораздо меньше ошибок.

При гибридной сборке разница в между использованием покрытиям Pacbio 40х и 80х отсутствует совсем, отличие от 20х предствляется незначительным: чуть короче самый длинный контиг (4636416 против 4641722 при 40х/80х) и ниже % покрытия (99.969 против 99.983). При этом, что примечательно, сборки 20x ниже частота ошибок.

Покрытие Pacbio 10x оказалось недостаточным для того, чтобы "полностью" собрать геном - самый длинный контиг сборки насчитывает 1460522 п.н.

Можно сделать вывод, что для получения сборки приемлемого качества достаточно покрытия 40х только длинными ридами, или покрытия 20х длинными ридами с использованием ридов Illumina для повышения точности.